In [1]:
import torch
import torch.nn as nn

class NARRE(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, hidden_dim, num_layers, word_embedding, kernel_sizes, num_filters, dropout_prob):
        super(NARRE, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.word_embedding = nn.Embedding.from_pretrained(torch.FloatTensor(word_embedding))
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.review_encoder = nn.GRU(embedding_dim, hidden_dim, num_layers, bidirectional=True, batch_first=True, dropout=dropout_prob)
        self.convolution = nn.ModuleList([nn.Conv2d(1, num_filters, (k, hidden_dim * 2), padding=(k-1, 0)) for k in kernel_sizes])
        self.attention = nn.Linear(num_filters * len(kernel_sizes), embedding_dim)
        self.output_layer = nn.Linear(embedding_dim, 1)

    def forward(self, user_ids, item_ids, reviews):
        user_embedded = self.user_embedding(user_ids)
        item_embedded = self.item_embedding(item_ids)
        review_embedded = self.word_embedding(reviews)
        review_encoded, _ = self.review_encoder(review_embedded)
        review_encoded = torch.transpose(review_encoded, 1, 2)
        review_convolved = [torch.relu(conv(review_encoded.unsqueeze(1))).squeeze(3) for conv in self.convolution]
        review_pooled = [torch.max_pool1d(conv, conv.size(2)).squeeze(2) for conv in review_convolved]
        review_representations = torch.cat(review_pooled, 1)
        review_attention = torch.softmax(self.attention(review_representations), dim=1)
        weighted_reviews = torch.mul(review_representations, review_attention.unsqueeze(2)).sum(dim=1)
        concatenated = torch.cat((user_embedded, item_embedded, weighted_reviews), dim=1)
        predicted_rating = self.output_layer(concatenated)
        return predicted_rating


In [2]:
# Importe as bibliotecas necessárias
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [3]:
# Defina a classe do conjunto de dados
class AmazonDataset(Dataset):
    def __init__(self, data_file):
        # Implemente a lógica de carregamento dos dados
        ...

    def __len__(self):
        # Implemente a lógica de retorno do tamanho do conjunto de dados
        ...

    def __getitem__(self, idx):
        # Implemente a lógica de retorno de um exemplo do conjunto de dados
        ...

In [4]:
# Defina as configurações de hiperparâmetros do modelo
vocab_size = ...
embedding_dim = ...
hidden_dim = ...
num_layers = ...
num_classes = ...
batch_size = ...
learning_rate = ...
num_epochs = ...

# Carregue o conjunto de dados
train_dataset = AmazonDataset(train_file)
val_dataset = AmazonDataset(val_file)
test_dataset = AmazonDataset(test_file)

NameError: name 'train_file' is not defined

In [ ]:
# Crie os dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Instancie o modelo NARRE
model = NARRE(vocab_size, embedding_dim, hidden_dim, num_layers, num_classes)

# Defina o otimizador e a função de perda
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Treine o modelo
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Implemente a lógica de treinamento do modelo
        ...

# Avalie o modelo
with torch.no_grad():
    for inputs, labels in test_loader:
        # Implemente a lógica de avaliação do modelo